# Azure Key Vault
Here you will create a key vault service. Subsequently a service principal will be created.
Based on this tutorial: https://microsoftlearning.github.io/mslearn-ai-services/Instructions/Exercises/02-ai-services-security.html

## Create a key vault and add a secret

First, you need to create a key vault and add a *secret* for the Azure AI services key.

1. Make a note of the **key1** value for your Azure AI services resource (or copy it to the clipboard).
2. In the Azure portal, on the **Home** page, select the **&#65291;Create a resource** button, search for *Key Vault*, and create a **Key Vault** resource with the following settings:

    - **Basics** tab
        - **Subscription**: *Your Azure subscription*
        - **Resource group**: *The same resource group as your Azure AI service resource*
        - **Key vault name**: *Enter a unique name*
        - **Region**: *The same region as your Azure AI service resource*
        - **Pricing tier**: Standard
    
    - **Access configuration** tab
        -  **Permission model**: Vault access policy
        - Scroll down to the **Access policies** section and select your user using the checkbox on the left. Then select **Review + create**, and select **Create** to create your resource.
     
3. Wait for deployment to complete and then go to your key vault resource.
4. In the left navigation pane, select **Secrets** (in the Objects section).
5. Select **+ Generate/Import** and add a new secret with the following settings :
    - **Upload options**: Manual
    - **Name**: AI-Services-Key *(it's important to match this exactly, because later you'll run code that retrieves the secret based on this name)*
    - **Value**: *Your **key1** Azure AI services key*
6. Select **Create**.

## Create a service principal for the key vault

In [8]:
# Use the CLI
import os
from dotenv import load_dotenv
load_dotenv()

subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_group_name = os.environ["RESOURCE_GROUP_NAME"]
sp_name = 'api://ai102-test-app'
key_vault_name = 'ai102-keys'

In [2]:
# Actually using cli commands in notebooks is dangerous, because it can expose secrets.
# So don't do this at work ;)
output = !az ad sp create-for-rbac -n {sp_name} --role owner --scopes subscriptions/{subscription_id}/resourceGroups/{resource_group_name}

In [3]:
# FYI: I'm trying to read out the parameters without exposing them 
out = "".join(output)
appId = out.split('"appId": ')[-1].split('"')[1]
password = out.split('"password": ')[-1].split('"')[1]
tenant = out.split('"tenant": ')[-1].split('"')[1]

In [14]:
# Next, get the object id
obj = !az ad sp show --id {appId}
obj_id = "".join(obj).split('"id": ')[-1].split('"')[1]

In [18]:
# Finally, assign permissions for the new service principal to access secrets in your key vault
permissions = !az keyvault set-policy -n {key_vault_name} --object-id {obj_id} --resource-group {resource_group_name} --secret-permissions get list

## Test the service principal

Check that your service principal can access the secret Azure AI services key in your key vault and use it to connect to your Azure AI services resource

In [24]:
# Add all necessary to the environment
os.environ["AI_SERVICE_ENDPOINT"] = "https://ai102-test.cognitiveservices.azure.com/"
os.environ["KEY_VAULT"] = key_vault_name
os.environ["TENANT_ID"] = tenant
os.environ["APP_ID"] = appId
os.environ["APP_PASSWORD"] = password

In [25]:
# import keyvault-client.py from the same directory
import keyvault_client
keyvault_client.main()

Language: English


Seems to be working! Have a look into the `keyvault_client.py` to better understand how the secrets are fetched from the Azure key vault using the service principal.